In [1]:
# Train generator and discriminator 
# directly on surface
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch3d.loss import (
    chamfer_distance,    
)

from src.callback.log_mesh import LogMesh
from src.utilities.util import grid_to_list
from src.models.discriminator import Discriminator
from src.models.surface_generator import Generator

In [2]:
class SurfaceSupervised(pl.LightningModule):
    
    def __init__(self, hparams):
        super().__init__()        
        self.save_hyperparameters(hparams)
        
        self.G = Generator(hparams)
        #self.D = Discriminator(hparams)
        
        
    def forward(self, outline):
        return self.G(outline)
    
    def log_all(self, r, p, d):
        for k, v in  d.items(): 
            self.log(f"{r}_loss_{p}/{k}", v)        
            
    def adversarial_loss(self, lbl, is_real):
        trg = torch.ones_like(lbl) if is_real else torch.zeros_like(lbl)
        return F.binary_cross_entropy_with_logits(lbl, trg)
    
    def train_generator(self, vertices, batch):
        loss = F.l1_loss(vertices, batch['baseline'])                         
        log =  {
            'G_loss' : loss.item(),            
        }        
        return loss, log    
    
    def training_step(self, batch, batch_idx):
        vertices = self.G(batch['outline'])
        loss, log = self.train_generator(vertices, batch)
        self.log_all('train', 'G', log)            
        return loss
        
#     def validation_step(self, batch, batch_idx):
#         vertices = self.G(batch['outline'])
#         _, log = self.train_discriminator(vertices, batch)        
#         self.log_all('val', 'D', log)   
#         _, log = self.train_generator(vertices, batch)
#         self.log_all('val', 'G', log)     
        
    def configure_optimizers(self):
        lr, betas = 0.0003, (0.5, 0.999)
        #opt_d = torch.optim.Adam(self.D.parameters(), lr=lr, betas=betas)
        opt_g = torch.optim.Adam(self.G.parameters(),  lr=lr, betas=betas)
        return [opt_g], []
    
    

from src.config import get_parser

config = get_parser().parse_args(args=[])   

config.log_mesh_interval = 100
config.fast_outline_size =  32
#config.fast_baseline_size = 128
#config.fast_image_size = 128
config.fast_batch_size = 16
config.raster_faces_per_pixel = 4
config.G_noise_amp = 0.005 #0.002
#config.geoaug_policy = 'scaling'
#config.fast_discriminator_channels[0] = 3


model = SurfaceSupervised(config)
model

SurfaceSupervised(
  (G): Generator(
    (points): SurfaceGenerator(
      (trunk): Sequential(
        (head): UpConvBlock(
          (upsample): Upsample(scale_factor=4.0, mode=bilinear)
          (conv): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (lrelu): LeakyReLU(negative_slope=0.2)
        )
        (main): Sequential(
          (b0): ConvBlock(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (lrelu): LeakyReLU(negative_slope=0.2)
          )
          (b1): ConvBlock(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (lrelu): LeakyReLU(negative_slope=0.2)
          )
          (b2): ConvBlock(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (lrelu): LeakyReLU(negative_slope=0.2)
          )
        )
      )
      (points): Sequential(
        (0):

In [3]:
from src.data.surface_datamodule import SurfaceDataModule

dm = SurfaceDataModule(config)    
dm

In [ ]:
trainer = pl.Trainer(gpus=1, max_epochs=1000, progress_bar_refresh_rate=5,
                     #terminate_on_nan=True, 
                     #profiler="pytorch",
                     log_every_n_steps=2, 
                     callbacks=[LogMesh(config)],
                     precision=16,
                    )
trainer.fit(model, dm)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type      | Params
-----------------------------------
0 | G    | Generator | 449 K 
-----------------------------------
449 K     Trainable params
0         Non-trainable params
449 K     Total params
1.797     Total estimated model params size (MB)
